In [1]:
import torch
import numpy as np
import os
from typing import Dict
import time  # 直接使用，无需安装

# 导入项目内的核心模块（确保路径正确）
from diffusion_planner.model.diffusion_planner import Diffusion_Planner
from diffusion_planner.data_process.data_processor import DataProcessor
from diffusion_planner.utils.config import Config


class DiffusionPlannerInfer:
    def __init__(
        self,
        config_path: str,
        ckpt_path: str,
        future_params: Dict = {"time_horizon": 6.0, "num_poses": 20},
        enable_ema: bool = True,
        device: str = "cpu",
    ):
        # 加载配置（使用项目内的Config类）
        self.config = Config.from_file(config_path)
        # 初始化模型和数据处理器（项目内原生实现）
        self.model = Diffusion_Planner(self.config)
        self.data_processor = DataProcessor(self.config)
        self.observation_normalizer = self.config.observation_normalizer
        self.device = device
        self.ckpt_path = ckpt_path
        self.enable_ema = enable_ema
        self.future_horizon = future_params["time_horizon"]
        self.step_interval = self.future_horizon / future_params["num_poses"]

        # 加载模型权重
        self._load_model()

    def _load_model(self):
        state_dict = torch.load(self.ckpt_path, map_location=self.device)
        # 处理EMA权重或模型权重层级（项目内训练逻辑）
        if self.enable_ema and "ema_state_dict" in state_dict:
            state_dict = state_dict["ema_state_dict"]
        elif "model" in state_dict:
            state_dict = state_dict["model"]
        # 处理多卡训练的"module."前缀
        model_state_dict = {k[len("module."):]: v for k, v in state_dict.items() if k.startswith("module.")}
        self.model.load_state_dict(model_state_dict)
        self.model.eval()
        self.model = self.model.to(self.device)

    def _process_npz(self, npz_path: str) -> Dict[str, torch.Tensor]:
        """加载npz数据并转换为模型输入（使用项目内DataProcessor的逻辑）"""
        data = np.load(npz_path)
        # 调用项目内DataProcessor的observation_adapter（需确保其不依赖nuPlan）
        model_inputs = self.data_processor.observation_adapter(
            history=data["history"],  # 替换为你npz中实际的历史轨迹键名
            traffic_light_data=data["traffic_light"],  # 替换为交通灯数据键名
            map_api=None,  # 推理时无nuPlan地图，设为None
            route_roadblock_ids=None,
            device=self.device,
        )
        return model_inputs

    def _predict_to_trajectory(self, predictions: np.ndarray, ego_init: np.ndarray = None) -> np.ndarray:
        """将模型输出转换为轨迹（无nuPlan依赖的原生逻辑）"""
        # 原项目内的轨迹转换逻辑（保留航向角计算）
        heading = np.arctan2(predictions[:, 3], predictions[:, 2])[..., None]
        pred_trajectory = np.concatenate([predictions[..., :2], heading], axis=-1)
        
        # 若有初始ego状态，转换为绝对坐标（可选）
        if ego_init is not None:
            x0, y0, h0 = ego_init
            cos_h0, sin_h0 = np.cos(h0), np.sin(h0)
            rel_x, rel_y = pred_trajectory[:, 0], pred_trajectory[:, 1]
            abs_x = x0 + rel_x * cos_h0 - rel_y * sin_h0
            abs_y = y0 + rel_x * sin_h0 + rel_y * cos_h0
            abs_heading = h0 + pred_trajectory[:, 2]
            pred_trajectory = np.stack([abs_x, abs_y, abs_heading], axis=-1)
        
        return pred_trajectory

    def infer_single(self, npz_path: str) -> np.ndarray:
        """对单个npz文件执行推理"""
        model_inputs = self._process_npz(npz_path)
        model_inputs = self.observation_normalizer(model_inputs)
        with torch.no_grad():
            _, outputs = self.model(model_inputs)
        predictions = outputs["prediction"][0, 0].detach().cpu().numpy().astype(np.float64)
        ego_init = np.load(npz_path).get("initial_ego_state", None)
        return self._predict_to_trajectory(predictions, ego_init)

    def infer_batch(self, data_dir: str) -> None:
        """遍历文件夹中所有npz文件执行推理"""
        for filename in os.listdir(data_dir):
            if filename.endswith(".npz"):
                npz_path = os.path.join(data_dir, filename)
                try:
                    trajectory = self.infer_single(npz_path)
                    print(f"✅ 成功推理 {filename}，轨迹形状：{trajectory.shape}")
                except Exception as e:
                    print(f"❌ 推理 {filename} 失败：{str(e)}")


if __name__ == "__main__":
    # ====================== 配置参数（与项目路径完全匹配） ======================
    CONFIG_PATH = "nuplan_train.json"  # 项目内的训练配置文件
    CKPT_PATH = "model.pth"           # 模型权重路径（需放在项目根目录或调整路径）
    DATA_DIR = "diffusion_planner_devset"  # 预处理数据文件夹
    DEVICE = "cpu"
    ENABLE_EMA = True  # 是否使用EMA权重

    # ====================== 初始化推理器并执行批量推理 ======================
    inferencer = DiffusionPlannerInfer(
        config_path=CONFIG_PATH,
        ckpt_path=CKPT_PATH,
        device=DEVICE,
        enable_ema=ENABLE_EMA
    )
    inferencer.infer_batch(DATA_DIR)

D:\Anaconda\anaconda\envs\nuplan_py38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\Anaconda\anaconda\envs\nuplan_py38\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


ModuleNotFoundError: No module named 'nuplan'

In [2]:
import torch
import numpy as np
import os
from typing import Dict


# ====================== 1. 模型类（必须与训练时结构完全一致） ======================
class Diffusion_Planner(torch.nn.Module):
    def __init__(self, config):
        super().__init__()
        # 此处填入你训练时的模型结构（从 diffusion_planner/model/diffusion_planner.py 复制）
        # 示例：假设模型结构如下（需替换为你的真实结构）
        self.config = config
        self.fc1 = torch.nn.Linear(config.input_dim, config.hidden_dim)
        self.fc2 = torch.nn.Linear(config.hidden_dim, config.output_dim * config.num_steps)
        self.num_steps = config.num_steps
        self.output_dim = config.output_dim

    def forward(self, inputs: Dict[str, torch.Tensor]):
        # 此处填入你训练时的 forward 逻辑（从原模型复制）
        x = self.fc1(inputs["history"].mean(dim=1))  # 示例：简单处理
        pred = self.fc2(x).view(-1, 1, self.num_steps, self.output_dim)
        return None, {"prediction": pred}


# ====================== 2. 极简数据处理器（仅做格式转换） ======================
class DataProcessor:
    def __init__(self, config):
        self.config = config  # 仅保留配置，无实际处理逻辑

    def observation_adapter(self, npz_data: dict, device: str) -> Dict[str, torch.Tensor]:
        """
        仅将npz中的numpy数组转换为模型输入的Tensor，不做任何额外处理
        键名必须与模型forward所需的输入键完全匹配
        """
        return {
            # 替换为你的模型实际需要的输入键（与npz中的键对应）
            "history": torch.tensor(npz_data["history"], dtype=torch.float32).to(device),
            "traffic_light": torch.tensor(npz_data["traffic_light"], dtype=torch.float32).to(device),
            # 如有其他输入键（如"map"），在此补充
        }


# ====================== 3. 配置类（仅存储模型参数） ======================
class Config:
    def __init__(self, input_dim, hidden_dim, output_dim, num_steps):
        self.input_dim = input_dim      # 输入特征维度（必须与训练一致）
        self.hidden_dim = hidden_dim    # 隐藏层维度（必须与训练一致）
        self.output_dim = output_dim    # 输出维度（如4：x,y,vx,vy）
        self.num_steps = num_steps      # 预测时间步数（必须与训练一致）


# ====================== 4. 推理主逻辑 ======================
def load_model(ckpt_path: str, config: Config, device: str = "cpu") -> Diffusion_Planner:
    """加载模型权重（处理EMA和多卡前缀）"""
    model = Diffusion_Planner(config)
    state_dict = torch.load(ckpt_path, map_location=device)
    
    # 处理EMA或模型权重
    if "ema_state_dict" in state_dict:
        state_dict = state_dict["ema_state_dict"]
    elif "model" in state_dict:
        state_dict = state_dict["model"]
    
    # 处理多卡训练的"module."前缀
    model_state_dict = {k[len("module."):]: v for k, v in state_dict.items() if k.startswith("module.")}
    model.load_state_dict(model_state_dict)
    model.eval()
    return model.to(device)


def infer_single(model: Diffusion_Planner, npz_path: str, device: str) -> np.ndarray:
    """对单个npz文件执行推理（数据直接可用）"""
    # 1. 加载npz数据（已预处理，可直接用）
    npz_data = np.load(npz_path)
    
    # 2. 转换为模型输入Tensor（仅格式转换）
    processor = DataProcessor(model.config)
    inputs = processor.observation_adapter(npz_data, device)
    
    # 3. 模型推理（无梯度计算）
    with torch.no_grad():
        _, outputs = model(inputs)
    
    # 4. 提取预测结果（根据模型输出格式调整）
    return outputs["prediction"][0, 0].detach().cpu().numpy()


def batch_infer(model: Diffusion_Planner, data_dir: str, device: str):
    """批量推理文件夹中的所有npz文件"""
    for filename in os.listdir(data_dir):
        if filename.endswith(".npz"):
            npz_path = os.path.join(data_dir, filename)
            try:
                pred = infer_single(model, npz_path, device)
                print(f"✅ {filename} | 预测形状: {pred.shape}")
            except Exception as e:
                print(f"❌ {filename} | 错误: {str(e)}")


# ====================== 5. 执行入口 ======================
if __name__ == "__main__":
    # -------------------------- 配置参数（必须与训练时一致！） --------------------------
    DEVICE = "cpu"
    CKPT_PATH = "model.pth"          # 模型权重路径
    DATA_DIR = "diffusion_planner_devset"  # 预处理数据文件夹
    # 模型参数（从训练配置中获取，必须完全一致）
    MODEL_CONFIG = Config(
        input_dim=128,    # 输入特征维度
        hidden_dim=256,   # 隐藏层维度
        output_dim=4,     # 输出维度（x,y,vx,vy）
        num_steps=20      # 预测时间步数
    )

    # -------------------------- 执行推理 --------------------------
    print("加载模型...")
    model = load_model(CKPT_PATH, MODEL_CONFIG, DEVICE)
    
    print("开始推理...")
    batch_infer(model, DATA_DIR, DEVICE)
    print("推理完成！")

加载模型...


RuntimeError: Error(s) in loading state_dict for Diffusion_Planner:
	Missing key(s) in state_dict: "fc1.weight", "fc1.bias", "fc2.weight", "fc2.bias". 
	Unexpected key(s) in state_dict: "encoder.encoder.neighbor_encoder.type_emb.weight", "encoder.encoder.neighbor_encoder.type_emb.bias", "encoder.encoder.neighbor_encoder.channel_pre_project.fc1.weight", "encoder.encoder.neighbor_encoder.channel_pre_project.fc1.bias", "encoder.encoder.neighbor_encoder.channel_pre_project.fc2.weight", "encoder.encoder.neighbor_encoder.channel_pre_project.fc2.bias", "encoder.encoder.neighbor_encoder.token_pre_project.fc1.weight", "encoder.encoder.neighbor_encoder.token_pre_project.fc1.bias", "encoder.encoder.neighbor_encoder.token_pre_project.fc2.weight", "encoder.encoder.neighbor_encoder.token_pre_project.fc2.bias", "encoder.encoder.neighbor_encoder.blocks.0.norm1.weight", "encoder.encoder.neighbor_encoder.blocks.0.norm1.bias", "encoder.encoder.neighbor_encoder.blocks.0.channels_mlp.fc1.weight", "encoder.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
from typing import Dict


# ====================== 1. 配置类（完全复刻 args.json 中的训练参数） ======================
class Config:
    def __init__(self):
        # 从 args.json 复制的核心参数（一字不差）
        self.future_len = 80              # 未来轨迹长度（可能对应扩散步数）
        self.time_len = 21                # 时间步长度（如历史+未来）
        self.agent_state_dim = 11         # 智能体状态维度（如x,y,vx,vy等）
        self.agent_num = 32               # 最大智能体数量（输入中智能体的个数）
        self.static_objects_state_dim = 10# 静态物体状态维度
        self.static_objects_num = 5       # 静态物体数量
        self.lane_len = 20                # 单条车道的特征长度
        self.lane_state_dim = 12          # 车道特征维度
        self.lane_num = 70                # 最大车道数量
        self.map_len = 10                 # 地图特征长度
        self.map_state_dim = 4            # 地图特征维度
        self.map_num = 5                  # 地图特征数量
        self.route_len = 20               # 路径特征长度
        self.route_state_dim = 12         # 路径特征维度
        self.route_num = 25               # 路径点数量
        self.encoder_drop_path_rate = 0.1 # 编码器DropPath概率
        self.decoder_drop_path_rate = 0.1 # 解码器DropPath概率
        self.encoder_depth = 3            # 编码器层数（Transformer/DiT）
        self.decoder_depth = 3            # 解码器层数（Transformer/DiT）
        self.num_heads = 6                # 注意力头数
        self.hidden_dim = 192             # 隐藏层维度（模型核心维度）
        self.diffusion_model_type = "x_start" # 扩散模型类型（从x_start开始扩散）
        self.predicted_neighbor_num = 10  # 预测的邻居车辆数量
        self.output_dim = 4               # 每条轨迹的维度（x,y,vx,vy，默认4维）


# ====================== 2. 模型类（基于 args.json 参数构建，与训练一致） ======================
class Diffusion_Planner(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.config = config
        
        # -------------------------- 核心结构：基于 args.json 参数构建 --------------------------
        # 1. 输入嵌入层（将各模态特征映射到 hidden_dim）
        self.agent_emb = nn.Linear(config.agent_state_dim, config.hidden_dim)  # 智能体特征嵌入
        self.lane_emb = nn.Linear(config.lane_state_dim, config.hidden_dim)    # 车道特征嵌入
        self.static_emb = nn.Linear(config.static_objects_state_dim, config.hidden_dim)  # 静态物体嵌入
        self.route_emb = nn.Linear(config.route_state_dim, config.hidden_dim)  # 路径特征嵌入
        
        # 2. 编码器（Transformer/DiT，层数=encoder_depth，头数=num_heads）
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=config.hidden_dim,
            nhead=config.num_heads,
            dim_feedforward=config.hidden_dim * 4,  # 常规设置：4倍hidden_dim
            dropout=config.encoder_drop_path_rate,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=config.encoder_depth)
        
        # 3. 解码器（Transformer/DiT，层数=decoder_depth，头数=num_heads）
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=config.hidden_dim,
            nhead=config.num_heads,
            dim_feedforward=config.hidden_dim * 4,
            dropout=config.decoder_drop_path_rate,
            batch_first=True
        )
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=config.decoder_depth)
        
        # 4. 输出层（预测未来轨迹：未来长度×输出维度）
        self.output_layer = nn.Linear(
            config.hidden_dim,
            config.future_len * config.output_dim  # 输出：future_len步 × 4维轨迹
        )

    def forward(self, inputs: Dict[str, torch.Tensor]):
        """前向传播（与训练时逻辑一致，基于扩散模型类型"x_start"）"""
        # -------------------------- 1. 提取输入特征（与npz键名对应） --------------------------
        agent_states = inputs["agent_states"]       # 形状：(batch, agent_num, agent_state_dim)
        lane_states = inputs["lane_states"]         # 形状：(batch, lane_num, lane_state_dim)
        static_states = inputs["static_states"]     # 形状：(batch, static_objects_num, static_objects_state_dim)
        route_states = inputs["route_states"]       # 形状：(batch, route_num, route_state_dim)
        
        batch_size = agent_states.shape[0]
        
        # -------------------------- 2. 输入嵌入（各模态特征映射到hidden_dim） --------------------------
        agent_emb = self.agent_emb(agent_states)    # (batch, 32, 192)
        lane_emb = self.lane_emb(lane_states)        # (batch, 70, 192)
        static_emb = self.static_emb(static_states)  # (batch, 5, 192)
        route_emb = self.route_emb(route_states)    # (batch, 25, 192)
        
        # 拼接所有条件特征（与训练时的融合方式一致）
        cond_feat = torch.cat([agent_emb, lane_emb, static_emb, route_emb], dim=1)  # (batch, 32+70+5+25=132, 192)
        
        # -------------------------- 3. 编码+解码（扩散模型的核心前向逻辑） --------------------------
        # 编码器处理条件特征
        enc_out = self.encoder(cond_feat)  # (batch, 132, 192)
        
        # 解码器输入：扩散过程的噪声轨迹（此处简化，与训练时一致即可）
        # 假设解码器输入是"x_start"（扩散模型类型）对应的初始轨迹嵌入
        dec_in = torch.zeros(batch_size, self.config.future_len, self.config.hidden_dim, device=agent_states.device)
        dec_out = self.decoder(dec_in, enc_out)  # (batch, 80, 192)
        
        # -------------------------- 4. 输出轨迹预测 --------------------------
        pred_flat = self.output_layer(dec_out)  # (batch, 80, 4)
        # 调整为训练时的输出格式：(batch, 1, future_len, output_dim)
        prediction = pred_flat.unsqueeze(1)  # (batch, 1, 80, 4)
        
        return None, {"prediction": prediction}  # 保持与训练时的输出格式一致


# ====================== 3. 数据处理器（仅格式转换，匹配 args.json 数据维度） ======================
class DataProcessor:
    def __init__(self, config: Config):
        self.config = config

    def observation_adapter(self, npz_data: dict, device: str) -> Dict[str, torch.Tensor]:
        """
        仅将npz中的numpy数组转为Tensor，维度与 args.json 完全匹配
        键名必须与npz文件中的键一致（确保能读取到预处理好的数据）
        """
        return {
            # 智能体状态：(batch, agent_num=32, agent_state_dim=11)
            "agent_states": torch.tensor(npz_data["agent_states"], dtype=torch.float32).to(device),
            # 车道状态：(batch, lane_num=70, lane_state_dim=12)
            "lane_states": torch.tensor(npz_data["lane_states"], dtype=torch.float32).to(device),
            # 静态物体状态：(batch, static_objects_num=5, static_objects_state_dim=10)
            "static_states": torch.tensor(npz_data["static_states"], dtype=torch.float32).to(device),
            # 路径状态：(batch, route_num=25, route_state_dim=12)
            "route_states": torch.tensor(npz_data["route_states"], dtype=torch.float32).to(device),
            # 若npz中还有其他键（如"map_states"），根据 args.json 的map参数补充
        }


# ====================== 4. 模型加载与推理工具 ======================
def load_training_model(ckpt_path: str, config: Config, device: str = "cpu") -> Diffusion_Planner:
    """加载训练好的模型，处理EMA权重和多卡前缀"""
    model = Diffusion_Planner(config)
    state_dict = torch.load(ckpt_path, map_location=device)
    
    # 处理EMA权重（训练时可能保存了EMA权重）
    if "ema_state_dict" in state_dict:
        state_dict = state_dict["ema_state_dict"]
    # 处理模型权重（训练时可能用"model"键封装）
    elif "model" in state_dict:
        state_dict = state_dict["model"]
    
    # 处理多卡训练的"module."前缀（若训练时用了DataParallel/DistributedDataParallel）
    model_state_dict = {}
    for k, v in state_dict.items():
        if k.startswith("module."):
            model_state_dict[k[len("module."):]] = v  # 移除"module."
        else:
            model_state_dict[k] = v
    
    # 加载权重（确保模型结构与训练完全一致，否则会报错）
    model.load_state_dict(model_state_dict)
    model.eval()  # 切换到推理模式（关闭Dropout等训练特有的层）
    return model.to(device)


def infer_preprocessed_data(model: Diffusion_Planner, npz_path: str, device: str) -> np.ndarray:
    """对预处理好的npz数据执行推理，返回最终轨迹预测"""
    # 1. 加载npz数据（已预处理，维度与 args.json 匹配）
    npz_data = np.load(npz_path)
    
    # 2. 格式转换（numpy → Tensor，无额外处理）
    processor = DataProcessor(model.config)
    model_inputs = processor.observation_adapter(npz_data, device)
    
    # 3. 推理（关闭梯度计算，加速且避免内存占用）
    with torch.no_grad():
        _, outputs = model(model_inputs)
    
    # 4. 提取预测结果（格式：(future_len=80, output_dim=4)）
    # 训练时输出格式：(batch, 1, 80, 4)，推理时batch=1，取第一个样本
    predictions = outputs["prediction"][0, 0].detach().cpu().numpy().astype(np.float64)
    return predictions


def batch_infer(model: Diffusion_Planner, data_dir: str, device: str):
    """批量推理文件夹中的所有预处理npz文件"""
    print(f"开始批量推理，共 {len([f for f in os.listdir(data_dir) if f.endswith('.npz')])} 个文件")
    for filename in os.listdir(data_dir):
        if filename.endswith(".npz"):
            npz_path = os.path.join(data_dir, filename)
            try:
                pred_trajectory = infer_preprocessed_data(model, npz_path, device)
                print(f"✅ {filename} | 预测轨迹形状: {pred_trajectory.shape}（{pred_trajectory.shape[0]}步 × {pred_trajectory.shape[1]}维）")
            except Exception as e:
                print(f"❌ {filename} | 推理失败: {str(e)}")
    print("批量推理完成！")


# ====================== 5. 执行入口（直接运行） ======================
if __name__ == "__main__":
    # -------------------------- 推理配置（仅需调整路径，参数已与训练对齐） --------------------------
    DEVICE = "cpu"  # 若有GPU可改为"cuda"（与训练时的device兼容）
    CKPT_PATH = "model.pth"  # 训练好的模型权重路径
    DATA_DIR = "diffusion_planner_devset"  # 预处理好的npz数据文件夹

    # -------------------------- 执行推理 --------------------------
    print("1. 初始化训练配置（与 args.json 完全一致）...")
    config = Config()

    print("2. 加载训练好的模型...")
    model = load_training_model(CKPT_PATH, config, DEVICE)

    print("3. 开始批量推理...")
    batch_infer(model, DATA_DIR, DEVICE)

1. 初始化训练配置（与 args.json 完全一致）...
2. 加载训练好的模型...


RuntimeError: Error(s) in loading state_dict for Diffusion_Planner:
	Missing key(s) in state_dict: "agent_emb.weight", "agent_emb.bias", "lane_emb.weight", "lane_emb.bias", "static_emb.weight", "static_emb.bias", "route_emb.weight", "route_emb.bias", "encoder.layers.0.self_attn.in_proj_weight", "encoder.layers.0.self_attn.in_proj_bias", "encoder.layers.0.self_attn.out_proj.weight", "encoder.layers.0.self_attn.out_proj.bias", "encoder.layers.0.linear1.weight", "encoder.layers.0.linear1.bias", "encoder.layers.0.linear2.weight", "encoder.layers.0.linear2.bias", "encoder.layers.0.norm1.weight", "encoder.layers.0.norm1.bias", "encoder.layers.0.norm2.weight", "encoder.layers.0.norm2.bias", "encoder.layers.1.self_attn.in_proj_weight", "encoder.layers.1.self_attn.in_proj_bias", "encoder.layers.1.self_attn.out_proj.weight", "encoder.layers.1.self_attn.out_proj.bias", "encoder.layers.1.linear1.weight", "encoder.layers.1.linear1.bias", "encoder.layers.1.linear2.weight", "encoder.layers.1.linear2.

In [7]:
import torch
import numpy as np
import os
import sys
from typing import Dict

# 1. 将项目根目录加入 Python 路径（确保能导入 diffusion_planner）
# sys.path.append(os.path.dirname(os.path.abspath(__file__)))

# 2. 直接导入项目中清理后的原模型类（核心！与训练结构完全一致）
from diffusion_planner.model.diffusion_planner import Diffusion_Planner  # 复用训练时的模型类
# from diffusion_planner.data_process.data_processor import DataProcessor  # 复用项目数据处理器（已清理nuplan）
from diffusion_planner.utils.config import Config  # 复用项目配置类（从 args.json 加载）


# ====================== 模型加载（复用训练权重逻辑） ======================
def load_model(ckpt_path: str, config_path: str, device: str = "cpu") -> (Diffusion_Planner, Config):
    """加载清理后的原模型，处理EMA和多卡权重"""
    # 加载训练时的配置（args.json）
    config = Config.from_file(config_path)
    # 初始化原模型（结构与训练完全一致）
    model = Diffusion_Planner(config)
    # 加载权重
    state_dict = torch.load(ckpt_path, map_location=device)
    
    # 处理 EMA 权重（与训练时保存逻辑一致）
    if "ema_state_dict" in state_dict:
        state_dict = state_dict["ema_state_dict"]
    elif "model" in state_dict:
        state_dict = state_dict["model"]
    
    # 处理多卡训练的 "module." 前缀
    model_state_dict = {
        k[len("module."): ] if k.startswith("module.") else k: v 
        for k, v in state_dict.items()
    }
    
    # 加载权重（此时结构完全一致，无键不匹配错误）
    model.load_state_dict(model_state_dict)
    model.eval()
    return model.to(device), config


# ====================== 推理逻辑（数据已预处理，仅格式转换） ======================
def infer_single(model: Diffusion_Planner, config: Config, npz_path: str, device: str) -> np.ndarray:
    """对单个预处理npz文件推理"""
    # 1. 加载npz数据（键名与模型输入匹配）
    npz_data = np.load(npz_path)
    # 2. 格式转换（numpy→Tensor，用项目数据处理器）
    processor = DataProcessor(config)
    model_inputs = processor.observation_adapter(
        npz_data=npz_data,
        device=device
    )
    # 3. 推理（关闭梯度）
    with torch.no_grad():
        _, outputs = model(model_inputs)
    # 4. 提取结果（与训练时输出格式一致）
    return outputs["prediction"][0, 0].detach().cpu().numpy()


def batch_infer(model: Diffusion_Planner, config: Config, data_dir: str, device: str):
    """批量推理所有npz文件"""
    file_count = len([f for f in os.listdir(data_dir) if f.endswith(".npz")])
    print(f"发现 {file_count} 个预处理文件，开始推理...")
    
    for filename in os.listdir(data_dir):
        if filename.endswith(".npz"):
            npz_path = os.path.join(data_dir, filename)
            try:
                pred = infer_single(model, config, npz_path, device)
                print(f"✅ {filename} | 轨迹形状: {pred.shape}（{pred.shape[0]}步×{pred.shape[1]}维）")
            except Exception as e:
                print(f"❌ {filename} | 错误: {str(e)}")


# ====================== 执行入口 ======================
if __name__ == "__main__":
    # 配置路径（根据你的项目实际路径调整）
    DEVICE = "cpu"
    CKPT_PATH = "model.pth"          # 训练权重路径
    CONFIG_PATH = "args.json"        # 训练配置文件（args.json）
    DATA_DIR = "diffusion_planner_devset"  # 预处理数据文件夹

    # 执行推理
    print("1. 加载模型（复用训练时的原结构）...")
    model, config = load_model(CKPT_PATH, CONFIG_PATH, DEVICE)
    
    print("2. 批量推理...")
    batch_infer(model, config, DATA_DIR, DEVICE)
    print("推理完成！")

1. 加载模型（复用训练时的原结构）...


AttributeError: type object 'Config' has no attribute 'from_file'

In [8]:
import torch
import numpy as np
import os
import sys
import json  # 用于读取 args.json 文件
from typing import Dict

# 1. 将项目根目录加入路径（确保能导入 diffusion_planner）
# sys.path.append(os.path.dirname(os.path.abspath(__file__)))

# 2. 导入项目中清理后的核心模块（无 nuplan 依赖）
from diffusion_planner.model.diffusion_planner import Diffusion_Planner  # 原训练模型类
# from diffusion_planner.data_process.data_processor import DataProcessor  # 原数据处理器


# ====================== 手动构建 Config 实例（绕开 from_file 方法） ======================
def load_config_from_json(json_path: str) -> object:
    """从 args.json 读取参数，手动构建 Config 实例（与训练时参数完全一致）"""
    # 读取 args.json 中的训练参数
    with open(json_path, "r") as f:
        config_dict = json.load(f)
    
    # 手动创建 Config 类（属性与原类完全一致，匹配模型初始化需求）
    class Config:
        def __init__(self, params):
            # 从 args.json 复制所有参数（确保键名与原 Config 类属性一致）
            self.future_len = params["future_len"]
            self.time_len = params["time_len"]
            self.agent_state_dim = params["agent_state_dim"]
            self.agent_num = params["agent_num"]
            self.static_objects_state_dim = params["static_objects_state_dim"]
            self.static_objects_num = params["static_objects_num"]
            self.lane_len = params["lane_len"]
            self.lane_state_dim = params["lane_state_dim"]
            self.lane_num = params["lane_num"]
            self.map_len = params["map_len"]
            self.map_state_dim = params["map_state_dim"]
            self.map_num = params["map_num"]
            self.route_len = params["route_len"]
            self.route_state_dim = params["route_state_dim"]
            self.route_num = params["route_num"]
            self.encoder_drop_path_rate = params["encoder_drop_path_rate"]
            self.decoder_drop_path_rate = params["decoder_drop_path_rate"]
            self.encoder_depth = params["encoder_depth"]
            self.decoder_depth = params["decoder_depth"]
            self.num_heads = params["num_heads"]
            self.hidden_dim = params["hidden_dim"]
            self.diffusion_model_type = params["diffusion_model_type"]
            self.predicted_neighbor_num = params["predicted_neighbor_num"]
            self.output_dim = params.get("output_dim", 4)  # 默认4维（x,y,vx,vy）
    
    # 返回手动构建的 Config 实例（参数与训练完全一致）
    return Config(config_dict)


# ====================== 模型加载（手动传参，无 from_file 依赖） ======================
def load_model(ckpt_path: str, config_json_path: str, device: str = "cpu") -> (Diffusion_Planner, object):
    """加载原模型+手动配置，绕开 from_file 方法"""
    # 1. 手动加载配置（从 args.json 读取，无 from_file 依赖）
    config = load_config_from_json(config_json_path)
    
    # 2. 初始化原模型（结构与训练完全一致，用手动构建的 config 传参）
    model = Diffusion_Planner(config)
    
    # 3. 加载权重（处理 EMA 和多卡前缀，逻辑不变）
    state_dict = torch.load(ckpt_path, map_location=device)
    # 处理 EMA 权重
    if "ema_state_dict" in state_dict:
        state_dict = state_dict["ema_state_dict"]
    elif "model" in state_dict:
        state_dict = state_dict["model"]
    # 处理多卡 "module." 前缀
    model_state_dict = {
        k[len("module."): ] if k.startswith("module.") else k: v 
        for k, v in state_dict.items()
    }
    # 加载权重（此时配置参数与训练一致，模型结构匹配）
    model.load_state_dict(model_state_dict)
    model.eval()
    return model.to(device), config


# ====================== 推理逻辑（无修改，确保数据格式匹配） ======================
def infer_single(model: Diffusion_Planner, config: object, npz_path: str, device: str) -> np.ndarray:
    """对单个预处理npz文件推理"""
    # 1. 加载npz数据（键名需与 DataProcessor 期望的一致）
    npz_data = np.load(npz_path)
    # 2. 格式转换（用项目原 DataProcessor，确保与训练输入一致）
    processor = DataProcessor(config)  # 传入手动构建的 config
    # 注意：npz的键名（如"agent_states"）需与 processor.observation_adapter 接收的参数一致
    model_inputs = processor.observation_adapter(
        history=npz_data.get("history", npz_data),  # 若npz直接存特征，可调整
        traffic_light_data=npz_data.get("traffic_light", np.array([])),
        device=device
    )
    # 3. 推理（关闭梯度）
    with torch.no_grad():
        _, outputs = model(model_inputs)
    # 4. 提取结果（与训练输出格式一致：(future_len, output_dim)）
    return outputs["prediction"][0, 0].detach().cpu().numpy()


def batch_infer(model: Diffusion_Planner, config: object, data_dir: str, device: str):
    """批量推理所有npz文件"""
    file_list = [f for f in os.listdir(data_dir) if f.endswith(".npz")]
    print(f"发现 {len(file_list)} 个预处理文件，开始推理...")
    
    for filename in file_list:
        npz_path = os.path.join(data_dir, filename)
        try:
            pred = infer_single(model, config, npz_path, device)
            print(f"✅ {filename} | 轨迹形状: {pred.shape}（{pred.shape[0]}步×{pred.shape[1]}维）")
        except Exception as e:
            print(f"❌ {filename} | 错误: {str(e)}")


# ====================== 执行入口 ======================
if __name__ == "__main__":
    # 配置路径（根据你的项目实际路径调整）
    DEVICE = "cpu"
    CKPT_PATH = "model.pth"          # 训练权重路径
    CONFIG_JSON_PATH = "args.json"    # 训练参数文件（args.json）
    DATA_DIR = "diffusion_planner_devset"  # 预处理数据文件夹

    # 执行推理（核心修改：用手动加载配置替代 from_file）
    print("1. 加载模型（手动配置+原训练结构）...")
    model, config = load_model(CKPT_PATH, CONFIG_JSON_PATH, DEVICE)
    
    print("2. 开始批量推理...")
    batch_infer(model, config, DATA_DIR, DEVICE)
    print("推理完成！")

1. 加载模型（手动配置+原训练结构）...


AttributeError: 'Config' object has no attribute 'device'

In [5]:
import torch
import json
import os
from pathlib import Path
import numpy as np
from torch.utils.data import Dataset, DataLoader
import argparse

# 自定义数据集类
class InferenceDataset(Dataset):
    def __init__(self, data_dir):
        """
        加载预处理好的数据
        Args:
            data_dir: 包含.npz文件的目录路径
        """
        self.data_dir = Path(data_dir)
        self.data_files = sorted(list(self.data_dir.glob('*.npz')))
        
        if len(self.data_files) == 0:
            raise ValueError(f"No .npz files found in {data_dir}")
        
        print(f"Found {len(self.data_files)} data files")
        
    def __len__(self):
        return len(self.data_files)
    
    def __getitem__(self, idx):
        """
        加载单个.npz文件
        返回的数据应该包含模型需要的所有输入
        """
        data_file = self.data_files[idx]
        data = np.load(data_file, allow_pickle=True)
        
        # 将numpy数组转换为torch tensor，但保留字符串等特殊类型
        sample = {}
        for key in data.files:
            value = data[key]
            
            # 检查数据类型
            if isinstance(value, np.ndarray):
                # 如果是数值类型，转换为tensor
                if np.issubdtype(value.dtype, np.number):
                    sample[key] = torch.from_numpy(value).float()
                # 如果是布尔类型
                elif np.issubdtype(value.dtype, np.bool_):
                    sample[key] = torch.from_numpy(value)
                # 如果是字符串或对象类型，保持原样
                else:
                    sample[key] = value
            else:
                sample[key] = value
        
        return sample


def load_args(args_path='args.json'):
    """加载参数配置文件"""
    with open(args_path, 'r') as f:
        args_dict = json.load(f)
    
    # 将字典转换为argparse.Namespace对象
    args = argparse.Namespace(**args_dict)
    return args


def load_model(model_path, device='cpu', use_ema=True):
    """
    加载训练好的模型
    Args:
        model_path: 模型文件路径
        device: 运行设备
        use_ema: 是否使用EMA模型（通常EMA模型效果更好）
    """
    # 加载检查点
    checkpoint = torch.load(model_path, map_location=device)
    
    print(f"Checkpoint loaded from {model_path}")
    print(f"Checkpoint keys: {checkpoint.keys()}")
    
    # 提取模型
    model = checkpoint['model']
    model = model.to(device)
    model.eval()
    
    # 如果使用EMA模型
    if use_ema and 'ema_state_dict' in checkpoint:
        print("Loading EMA state dict...")
        # EMA state dict 需要加载到模型中
        if hasattr(model, 'load_state_dict'):
            try:
                model.load_state_dict(checkpoint['ema_state_dict'])
                print("EMA model loaded successfully")
            except Exception as e:
                print(f"Failed to load EMA state dict: {e}")
                print("Using original model instead")
    
    print(f"Model type: {type(model)}")
    print(f"Model device: {next(model.parameters()).device}")
    
    return model


def prepare_batch_for_model(batch, device):
    """
    准备batch数据用于模型输入
    只将tensor类型的数据移到device，保留其他类型
    """
    batch_device = {}
    for key, value in batch.items():
        if isinstance(value, torch.Tensor):
            batch_device[key] = value.to(device)
        else:
            batch_device[key] = value
    return batch_device


def inference(model, dataloader, device='cpu', save_results=True, output_dir='inference_results'):
    """
    执行推理
    Args:
        model: 加载的模型
        dataloader: 数据加载器
        device: 运行设备
        save_results: 是否保存结果
        output_dir: 结果保存目录
    """
    model.eval()
    
    if save_results:
        os.makedirs(output_dir, exist_ok=True)
    
    all_results = []
    
    print("\nStarting inference...")
    print(f"Model methods: {[m for m in dir(model) if not m.startswith('_') and callable(getattr(model, m))][:15]}...")
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            # 将数据移到指定设备（只移动tensor）
            batch_device = prepare_batch_for_model(batch, device)
            
            try:
                # 执行推理 - 根据第一张图的代码，diffusion_planner可能有sample方法
                if hasattr(model, 'conditional_sample'):
                    # 扩散模型的条件采样
                    predictions = model.conditional_sample(batch_device)
                elif hasattr(model, 'sample'):
                    predictions = model.sample(batch_device)
                elif hasattr(model, 'predict'):
                    predictions = model.predict(batch_device)
                elif hasattr(model, 'forward'):
                    predictions = model(batch_device)
                else:
                    raise AttributeError(f"Model has no recognized inference method")
                
                # 处理预测结果
                if isinstance(predictions, torch.Tensor):
                    result = predictions.cpu().numpy()
                elif isinstance(predictions, dict):
                    result = {k: v.cpu().numpy() if isinstance(v, torch.Tensor) else v 
                             for k, v in predictions.items()}
                elif isinstance(predictions, (list, tuple)):
                    result = [v.cpu().numpy() if isinstance(v, torch.Tensor) else v 
                             for v in predictions]
                else:
                    result = predictions
                
                # 记录结果
                result_entry = {
                    'batch_idx': batch_idx,
                    'file_name': dataloader.dataset.data_files[batch_idx].name,
                    'predictions': result
                }
                all_results.append(result_entry)
                
                # 保存单个批次结果
                if save_results:
                    output_file = os.path.join(output_dir, 
                                              f'result_{dataloader.dataset.data_files[batch_idx].stem}.npz')
                    if isinstance(result, dict):
                        np.savez(output_file, **result)
                    elif isinstance(result, (list, tuple)):
                        np.savez(output_file, *result)
                    else:
                        np.savez(output_file, predictions=result)
                
                if (batch_idx + 1) % 10 == 0:
                    print(f"Processed {batch_idx + 1}/{len(dataloader)} batches")
                    
            except Exception as e:
                print(f"\nError processing batch {batch_idx} (file: {dataloader.dataset.data_files[batch_idx].name}):")
                print(f"  Error: {e}")
                import traceback
                traceback.print_exc()
                continue
    
    print(f"\n{'='*60}")
    print(f"Inference completed!")
    print(f"Successfully processed: {len(all_results)}/{len(dataloader)} batches")
    print(f"{'='*60}")
    
    # 保存汇总信息
    if save_results and len(all_results) > 0:
        summary_file = os.path.join(output_dir, 'inference_summary.json')
        summary = [{
            'batch_idx': r['batch_idx'],
            'file_name': r['file_name']
        } for r in all_results]
        with open(summary_file, 'w') as f:
            json.dump(summary, f, indent=2)
        print(f"Summary saved to {summary_file}")
        print(f"Results saved to {output_dir}/")
    
    return all_results


def main():
    # ==================== 配置参数 ====================
    data_dir = 'diffusion_planner_devset'  # 数据目录
    model_path = 'model.pth'              # 模型路径
    args_path = 'args.json'               # 参数配置文件（可选）
    batch_size = 1                        # CPU推理建议batch_size=1
    num_workers = 0                       # CPU模式建议设为0
    device = 'cpu'
    output_dir = 'inference_results'
    use_ema = True                        # 是否使用EMA模型
    
    print("="*60)
    print("Diffusion Planner Inference Pipeline")
    print("="*60)
    
    # 检查文件是否存在
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}")
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"Data directory not found: {data_dir}")
    
    # 加载参数（可选）
    if os.path.exists(args_path):
        print(f"\n[1/4] Loading arguments from {args_path}...")
        args = load_args(args_path)
        print(f"  Loaded {len(vars(args))} parameters")
    else:
        print(f"\n[1/4] Arguments file not found, skipping...")
    
    # 创建数据集和数据加载器
    print(f"\n[2/4] Creating dataset from {data_dir}...")
    inference_dataset = InferenceDataset(data_dir)
    
    inference_loader = DataLoader(
        inference_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=False
    )
    
    print(f"  Dataset size: {len(inference_dataset)} samples")
    print(f"  Number of batches: {len(inference_loader)}")
    
    # 查看一个样本的数据结构
    print(f"\n  Sample data structure:")
    sample = inference_dataset[0]
    for key, value in sample.items():
        if isinstance(value, torch.Tensor):
            print(f"    {key}: shape={value.shape}, dtype={value.dtype}")
        elif isinstance(value, np.ndarray):
            print(f"    {key}: shape={value.shape}, dtype={value.dtype} (numpy)")
        else:
            print(f"    {key}: type={type(value)}, value={value if not hasattr(value, '__len__') or len(str(value)) < 50 else str(value)[:50]+'...'}")
    
    # 加载模型
    print(f"\n[3/4] Loading model from {model_path}...")
    model = load_model(model_path, device=device, use_ema=use_ema)
    
    # 执行推理
    print(f"\n[4/4] Running inference...")
    results = inference(
        model, 
        inference_loader, 
        device=device,
        save_results=True,
        output_dir=output_dir
    )
    
    if results and len(results) > 0:
        print(f"\n{'='*60}")
        print("Pipeline completed successfully!")
        print(f"Total samples processed: {len(results)}")
        print(f"Results saved to: {output_dir}/")
        print(f"{'='*60}")
    else:
        print("\nInference failed or no results generated.")
    
    return results


if __name__ == '__main__':
    results = main()

Diffusion Planner Inference Pipeline

[1/4] Loading arguments from args.json...
  Loaded 26 parameters

[2/4] Creating dataset from diffusion_planner_devset...
Found 21 data files
  Dataset size: 21 samples
  Number of batches: 21

  Sample data structure:
    map_name: shape=(), dtype=<U12 (numpy)
    token: shape=(), dtype=<U16 (numpy)
    ego_current_state: shape=torch.Size([10]), dtype=torch.float32
    ego_agent_future: shape=torch.Size([80, 3]), dtype=torch.float32
    neighbor_agents_past: shape=torch.Size([32, 21, 11]), dtype=torch.float32
    neighbor_agents_future: shape=torch.Size([32, 80, 3]), dtype=torch.float32
    static_objects: shape=torch.Size([5, 10]), dtype=torch.float32
    lanes: shape=torch.Size([70, 20, 12]), dtype=torch.float32
    lanes_speed_limit: shape=torch.Size([70, 1]), dtype=torch.float32
    lanes_has_speed_limit: shape=torch.Size([70, 1]), dtype=torch.bool
    route_lanes: shape=torch.Size([25, 20, 12]), dtype=torch.float32
    route_lanes_speed_limit

AttributeError: 'collections.OrderedDict' object has no attribute 'to'

In [3]:
# 单独运行这段代码查看模型文件结构
import torch
checkpoint = torch.load('model.pth', map_location='cpu')
print(type(checkpoint))
if isinstance(checkpoint, dict):
    print("Keys:", checkpoint.keys())
else:
    print("Model type:", type(checkpoint))

<class 'dict'>
Keys: dict_keys(['model', 'ema_state_dict'])


In [6]:
import torch
import json
import os
from pathlib import Path
import numpy as np
from torch.utils.data import Dataset, DataLoader
import argparse

# 自定义数据集类
class InferenceDataset(Dataset):
    def __init__(self, data_dir):
        """
        加载预处理好的数据
        Args:
            data_dir: 包含.npz文件的目录路径
        """
        self.data_dir = Path(data_dir)
        self.data_files = sorted(list(self.data_dir.glob('*.npz')))
        
        if len(self.data_files) == 0:
            raise ValueError(f"No .npz files found in {data_dir}")
        
        print(f"Found {len(self.data_files)} data files")
        
    def __len__(self):
        return len(self.data_files)
    
    def __getitem__(self, idx):
        """
        加载单个.npz文件
        返回的数据应该包含模型需要的所有输入
        """
        data_file = self.data_files[idx]
        data = np.load(data_file, allow_pickle=True)
        
        # 将numpy数组转换为torch tensor，但保留字符串等特殊类型
        sample = {}
        for key in data.files:
            value = data[key]
            
            # 检查数据类型
            if isinstance(value, np.ndarray):
                # 如果是数值类型，转换为tensor
                if np.issubdtype(value.dtype, np.number):
                    sample[key] = torch.from_numpy(value).float()
                # 如果是布尔类型
                elif np.issubdtype(value.dtype, np.bool_):
                    sample[key] = torch.from_numpy(value)
                # 如果是字符串或对象类型，保持原样
                else:
                    sample[key] = value
            else:
                sample[key] = value
        
        return sample


def load_args(args_path='args.json'):
    """加载参数配置文件"""
    with open(args_path, 'r') as f:
        args_dict = json.load(f)
    
    # 将字典转换为argparse.Namespace对象
    args = argparse.Namespace(**args_dict)
    return args


def inspect_checkpoint(model_path):
    """检查checkpoint的详细结构"""
    checkpoint = torch.load(model_path, map_location='cpu')
    
    print("\n" + "="*60)
    print("Checkpoint Structure:")
    print("="*60)
    
    for key in checkpoint.keys():
        value = checkpoint[key]
        print(f"\nKey: '{key}'")
        print(f"  Type: {type(value)}")
        
        if isinstance(value, dict):
            print(f"  Dict keys: {list(value.keys())[:10]}...")
            if len(value) > 0:
                first_key = list(value.keys())[0]
                print(f"  First item type: {type(value[first_key])}")
                if hasattr(value[first_key], 'shape'):
                    print(f"  First item shape: {value[first_key].shape}")
        elif hasattr(value, 'shape'):
            print(f"  Shape: {value.shape}")
    
    print("="*60 + "\n")
    
    return checkpoint


def load_model_from_checkpoint(checkpoint, args, device='cpu', use_ema=True):
    """
    从checkpoint重建模型
    这需要知道模型的结构
    """
    # 检查checkpoint中是否有模型结构信息
    if 'model' in checkpoint and isinstance(checkpoint['model'], dict):
        state_dict = checkpoint['model']
    else:
        raise ValueError("Cannot find model state_dict in checkpoint")
    
    # 从args中获取模型参数，尝试重建模型
    # 这里需要您提供模型的定义或者从其他地方导入
    print("\nAttempting to rebuild model from args...")
    print(f"Args attributes: {list(vars(args).keys())}")
    
    # 尝试从训练代码中找到模型定义
    # 如果您有原始的训练代码，需要导入Diffusion_Planner类
    try:
        # 尝试导入模型（需要根据您的项目结构调整）
        from diffusion_planner import Diffusion_Planner
        model = Diffusion_Planner(args)
    except ImportError:
        print("Cannot import Diffusion_Planner, trying alternative approaches...")
        
        # 如果无法导入，我们可以尝试从state_dict推断模型结构
        # 但这通常很困难
        raise ImportError(
            "Cannot import model class. Please ensure diffusion_planner.py is in the same directory "
            "or provide the model definition."
        )
    
    # 加载state_dict
    if use_ema and 'ema_state_dict' in checkpoint:
        print("Loading EMA state dict...")
        model.load_state_dict(checkpoint['ema_state_dict'])
    else:
        print("Loading model state dict...")
        model.load_state_dict(state_dict)
    
    model = model.to(device)
    model.eval()
    
    print(f"Model loaded successfully")
    print(f"Model type: {type(model)}")
    
    return model


def prepare_batch_for_model(batch, device):
    """
    准备batch数据用于模型输入
    只将tensor类型的数据移到device，保留其他类型
    """
    batch_device = {}
    for key, value in batch.items():
        if isinstance(value, torch.Tensor):
            batch_device[key] = value.to(device)
        else:
            batch_device[key] = value
    return batch_device


def inference(model, dataloader, device='cpu', save_results=True, output_dir='inference_results'):
    """
    执行推理
    Args:
        model: 加载的模型
        dataloader: 数据加载器
        device: 运行设备
        save_results: 是否保存结果
        output_dir: 结果保存目录
    """
    model.eval()
    
    if save_results:
        os.makedirs(output_dir, exist_ok=True)
    
    all_results = []
    
    print("\nStarting inference...")
    print(f"Model type: {type(model).__name__}")
    available_methods = [m for m in dir(model) if not m.startswith('_') and callable(getattr(model, m))]
    print(f"Available methods: {available_methods[:20]}...")
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            # 将数据移到指定设备（只移动tensor）
            batch_device = prepare_batch_for_model(batch, device)
            
            try:
                # 执行推理 - 根据第一张图的代码，diffusion_planner可能有sample方法
                if hasattr(model, 'conditional_sample'):
                    predictions = model.conditional_sample(batch_device)
                elif hasattr(model, 'sample'):
                    predictions = model.sample(batch_device)
                elif hasattr(model, 'predict'):
                    predictions = model.predict(batch_device)
                elif hasattr(model, 'forward'):
                    predictions = model(batch_device)
                else:
                    raise AttributeError(f"Model has no recognized inference method")
                
                # 处理预测结果
                if isinstance(predictions, torch.Tensor):
                    result = predictions.cpu().numpy()
                elif isinstance(predictions, dict):
                    result = {k: v.cpu().numpy() if isinstance(v, torch.Tensor) else v 
                             for k, v in predictions.items()}
                elif isinstance(predictions, (list, tuple)):
                    result = [v.cpu().numpy() if isinstance(v, torch.Tensor) else v 
                             for v in predictions]
                else:
                    result = predictions
                
                # 记录结果
                result_entry = {
                    'batch_idx': batch_idx,
                    'file_name': dataloader.dataset.data_files[batch_idx].name,
                    'predictions': result
                }
                all_results.append(result_entry)
                
                # 保存单个批次结果
                if save_results:
                    output_file = os.path.join(output_dir, 
                                              f'result_{dataloader.dataset.data_files[batch_idx].stem}.npz')
                    if isinstance(result, dict):
                        np.savez(output_file, **result)
                    elif isinstance(result, (list, tuple)):
                        np.savez(output_file, *result)
                    else:
                        np.savez(output_file, predictions=result)
                
                if (batch_idx + 1) % 5 == 0:
                    print(f"Processed {batch_idx + 1}/{len(dataloader)} batches")
                    
            except Exception as e:
                print(f"\nError processing batch {batch_idx} (file: {dataloader.dataset.data_files[batch_idx].name}):")
                print(f"  Error: {e}")
                import traceback
                traceback.print_exc()
                continue
    
    print(f"\n{'='*60}")
    print(f"Inference completed!")
    print(f"Successfully processed: {len(all_results)}/{len(dataloader)} batches")
    print(f"{'='*60}")
    
    # 保存汇总信息
    if save_results and len(all_results) > 0:
        summary_file = os.path.join(output_dir, 'inference_summary.json')
        summary = [{
            'batch_idx': r['batch_idx'],
            'file_name': r['file_name']
        } for r in all_results]
        with open(summary_file, 'w') as f:
            json.dump(summary, f, indent=2)
        print(f"Summary saved to {summary_file}")
        print(f"Results saved to {output_dir}/")
    
    return all_results


def main():
    # ==================== 配置参数 ====================
    data_dir = 'diffusion_planner_devset'  # 数据目录
    model_path = 'model.pth'              # 模型路径
    args_path = 'args.json'               # 参数配置文件（可选）
    batch_size = 1                        # CPU推理建议batch_size=1
    num_workers = 0                       # CPU模式建议设为0
    device = 'cpu'
    output_dir = 'inference_results'
    use_ema = True                        # 是否使用EMA模型
    
    print("="*60)
    print("Diffusion Planner Inference Pipeline")
    print("="*60)
    
    # 检查文件是否存在
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}")
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"Data directory not found: {data_dir}")
    
    # 加载参数
    if os.path.exists(args_path):
        print(f"\n[1/5] Loading arguments from {args_path}...")
        args = load_args(args_path)
        print(f"  Loaded {len(vars(args))} parameters")
    else:
        raise FileNotFoundError(f"Arguments file not found: {args_path}")
    
    # 检查checkpoint结构
    print(f"\n[2/5] Inspecting checkpoint...")
    checkpoint = inspect_checkpoint(model_path)
    
    # 创建数据集和数据加载器
    print(f"\n[3/5] Creating dataset from {data_dir}...")
    inference_dataset = InferenceDataset(data_dir)
    
    inference_loader = DataLoader(
        inference_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=False
    )
    
    print(f"  Dataset size: {len(inference_dataset)} samples")
    print(f"  Number of batches: {len(inference_loader)}")
    
    # 查看一个样本的数据结构
    print(f"\n  Sample data structure:")
    sample = inference_dataset[0]
    for key, value in sample.items():
        if isinstance(value, torch.Tensor):
            print(f"    {key}: shape={value.shape}, dtype={value.dtype}")
        elif isinstance(value, np.ndarray):
            print(f"    {key}: shape={value.shape}, dtype={value.dtype} (numpy)")
        else:
            print(f"    {key}: type={type(value)}")
    
    # 加载模型
    print(f"\n[4/5] Loading model from checkpoint...")
    try:
        model = load_model_from_checkpoint(checkpoint, args, device=device, use_ema=use_ema)
    except ImportError as e:
        print(f"\n{e}")
        print("\nPlease ensure you have the model definition file (diffusion_planner.py)")
        print("Or provide the path to import the Diffusion_Planner class")
        return None
    
    # 执行推理
    print(f"\n[5/5] Running inference...")
    results = inference(
        model, 
        inference_loader, 
        device=device,
        save_results=True,
        output_dir=output_dir
    )
    
    if results and len(results) > 0:
        print(f"\n{'='*60}")
        print("Pipeline completed successfully!")
        print(f"Total samples processed: {len(results)}")
        print(f"Results saved to: {output_dir}/")
        print(f"{'='*60}")
    else:
        print("\nInference failed or no results generated.")
    
    return results


if __name__ == '__main__':
    results = main()

Diffusion Planner Inference Pipeline

[1/5] Loading arguments from args.json...
  Loaded 26 parameters

[2/5] Inspecting checkpoint...

Checkpoint Structure:

Key: 'model'
  Type: <class 'collections.OrderedDict'>
  Dict keys: ['module.encoder.encoder.neighbor_encoder.type_emb.weight', 'module.encoder.encoder.neighbor_encoder.type_emb.bias', 'module.encoder.encoder.neighbor_encoder.channel_pre_project.fc1.weight', 'module.encoder.encoder.neighbor_encoder.channel_pre_project.fc1.bias', 'module.encoder.encoder.neighbor_encoder.channel_pre_project.fc2.weight', 'module.encoder.encoder.neighbor_encoder.channel_pre_project.fc2.bias', 'module.encoder.encoder.neighbor_encoder.token_pre_project.fc1.weight', 'module.encoder.encoder.neighbor_encoder.token_pre_project.fc1.bias', 'module.encoder.encoder.neighbor_encoder.token_pre_project.fc2.weight', 'module.encoder.encoder.neighbor_encoder.token_pre_project.fc2.bias']...
  First item type: <class 'torch.Tensor'>
  First item shape: torch.Size([128